In [50]:
# VideoFileClip 은 비디오 파일을 읽어올 때 사용되는 클래스 
from moviepy.editor import VideoFileClip
# ipython_display() 는 동영상을 주피터 노트북에 렌더링할 수 있게 도와주는 함수
from moviepy.editor import ipython_display

In [51]:
import cv2

#동영상 파일 열기 
vc = cv2.VideoCapture('love2.mp4')

# vc의 get() 함수로 읽을 수 있습니다. FRAME_COUNT 은 비디오 전체 프레임 개수
vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print (vlen) # video length

for i in range(vlen):
    # ret 은 read() 함수에서 이미지가 반환되면 True, 반대의 경우 False
    ret, img = vc.read()
    if ret == False:
        break
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('Face Detection', cv2.resize(img, None, fx=1.2, fy=1.2))
    # waitKey() 함수는 키보드가 입력될 때 키보드 값을 반환 27=esc
    if cv2.waitKey(1) == 27:
            break

            
vc.release() 
cv2.destroyAllWindows() #열린 창을 모두 닫음
 

54


In [52]:
# addsticker.py
import dlib
import cv2

def img2sticker_orig(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    dlib_rects = detector_hog(img_rgb, 0)
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)

    # head coord
    for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
        x = landmark[2][0] 
        y = landmark[2][1] //2
        w = dlib_rect.width()
        h = dlib_rect.width()
        break

    # sticker
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)
    #스티커 붙일 위치 
    refined_x = x +10
    refined_y = y - h //2 +100

    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0

    img_bgr = img_orig.copy()
    #sticker_area는 번역 이미지에서 스티커를 적용할 수 있는 위치
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]
    #스티커 이미지 횐색 배경부분 제거
    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
    np.where(img_sticker==255,sticker_area,img_sticker).astype(np.uint8)
    return img_bgr


detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


In [53]:
detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

vc = cv2.VideoCapture('love2.mp4')
img_sticker = cv2.imread('cat.png')


vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print (vlen) # 비디오 프레임의 총 개수

54


In [54]:
for i in range(vlen):
    ret, img = vc.read()
    if ret == False:
        break

    ## 추가된 부분
    start = cv2.getTickCount()
    img_result = img2sticker_orig(img, img_sticker.copy(), detector_hog, landmark_predictor)
    time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
    print ('[INFO] time: %.2fms'%time)

    cv2.imshow('show', img_result)
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()

[INFO] time: 138.24ms
[INFO] time: 120.56ms
[INFO] time: 98.77ms
[INFO] time: 75.08ms
[INFO] time: 78.34ms
[INFO] time: 119.79ms
[INFO] time: 113.01ms
[INFO] time: 91.89ms
[INFO] time: 125.75ms
[INFO] time: 89.76ms
[INFO] time: 98.80ms
[INFO] time: 91.16ms
[INFO] time: 84.41ms
[INFO] time: 82.23ms
[INFO] time: 124.42ms
[INFO] time: 123.86ms
[INFO] time: 123.37ms
[INFO] time: 131.09ms
[INFO] time: 125.65ms
[INFO] time: 120.09ms
[INFO] time: 106.25ms
[INFO] time: 141.70ms
[INFO] time: 239.79ms
[INFO] time: 116.72ms
[INFO] time: 84.22ms
[INFO] time: 115.80ms
[INFO] time: 101.04ms
[INFO] time: 120.19ms
[INFO] time: 133.21ms
[INFO] time: 90.77ms
[INFO] time: 125.79ms
[INFO] time: 120.42ms
[INFO] time: 123.86ms
[INFO] time: 115.08ms
[INFO] time: 113.96ms
[INFO] time: 118.80ms
[INFO] time: 122.11ms
[INFO] time: 132.37ms
[INFO] time: 129.75ms
[INFO] time: 83.49ms
[INFO] time: 151.69ms
[INFO] time: 113.02ms
[INFO] time: 91.75ms
[INFO] time: 108.60ms
[INFO] time: 119.25ms
[INFO] time: 84.76ms
[I

In [56]:
# savevideo.py

# 모델 데이터 가져오기
#!wget http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2

# 압축 풀기 
#!bzip2 -d /content/shape_predictor_5_face_landmarks.dat.bz2

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

vc = cv2.VideoCapture('love2.mp4')
img_sticker = cv2.imread('cat.png')

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print (vlen) # 비디오 프레임의 총 개수

# writer 초기화
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter('result_nose.mp4', fourcc, 30, (1280, 720))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False:
        break

    start = cv2.getTickCount()
    img_result = img2sticker_orig(img, img_sticker.copy(), detector_hog, landmark_predictor)
    time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
    print ('[INFO] time: %.2fms'%time)

        # 매 프레임 마다 저장합니다.    
    vw.write(cv2.resize(img_result, (1280,720)))

    cv2.imshow('show', img_result)
    key = cv2.waitKey(1)
    if key == 27:
        break

vw.release()
cv2.destroyAllWindows()

54
[INFO] time: 136.16ms
[INFO] time: 169.32ms
[INFO] time: 120.19ms
[INFO] time: 133.81ms
[INFO] time: 121.40ms
[INFO] time: 81.94ms
[INFO] time: 100.76ms
[INFO] time: 114.61ms
[INFO] time: 114.27ms
[INFO] time: 96.54ms
[INFO] time: 85.74ms
[INFO] time: 127.94ms
[INFO] time: 96.26ms
[INFO] time: 88.69ms
[INFO] time: 120.11ms
[INFO] time: 126.89ms
[INFO] time: 90.23ms
[INFO] time: 91.85ms
[INFO] time: 240.16ms
[INFO] time: 91.16ms
[INFO] time: 90.49ms
[INFO] time: 133.11ms
[INFO] time: 135.04ms
[INFO] time: 106.80ms
[INFO] time: 130.86ms
[INFO] time: 130.52ms
[INFO] time: 112.29ms
[INFO] time: 119.82ms
[INFO] time: 124.60ms
[INFO] time: 123.82ms
[INFO] time: 110.87ms
[INFO] time: 111.06ms
[INFO] time: 135.93ms
[INFO] time: 126.12ms
[INFO] time: 138.55ms
[INFO] time: 109.77ms
[INFO] time: 109.81ms
[INFO] time: 124.27ms
[INFO] time: 106.59ms
[INFO] time: 126.77ms
[INFO] time: 126.56ms
[INFO] time: 90.58ms
[INFO] time: 117.73ms
[INFO] time: 125.64ms
[INFO] time: 131.68ms
[INFO] time: 108.